## Fact Orders

**Data Reading**

In [0]:
df=spark.sql('select * from databricks_cata.silver.orders_silver')

In [0]:
df.display()

In [0]:
dim_customer=spark.sql('select DimCustomerKey,customer_id as dim_customer_id from databricks_cata.gold.dimcustomers')

dim_products=spark.sql('select product_id as DimProductKey,product_id as dim_product_id from databricks_cata.gold.dimproducts')

**Fact DataFrame**

In [0]:
df_fact=df.join(dim_customer,df.customer_id==dim_customer.dim_customer_id,'left').join(dim_products,df.product_id==dim_products.DimProductKey,'left')

df_fact_new=df_fact.drop('dim_customer_id','dim_product_id','customer_id','product_id')

## Upset logic

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('databricks_cata.gold.Factorders'):
  dlt_obj=DeltaTable.forName(spark,'databricks_cata.gold.Factorders')
  dlt_obj.alias('t').merge(df_fact_new.alias('s'),'t.order_id=s.order_id').whenMatchedUpdateAll().\
  whenNotMatchedInsertAll().execute()
else:
    df_fact_new.write.format('delta').option('path','abfss://gold2026@storagespotifyproject20.dfs.core.windows.net/FactOrders').saveAsTable('databricks_cata.gold.Factorders')


In [0]:
%sql
select count(*) from databricks_cata.gold.Factorders